# Aviation Data Cleaning and Analysis

## Introduction

In this notebook, I shall focus on cleaning and analyzing data from the `AviationData.csv` dataset. This particular dataset was provided by the National Safety and Transport Board and it contains information about aviation accidents from the year 1962 to the year 2023. The goal is to identify patterns, clean missing values, eliminate outliers, and prepare the data for further analysis or modeling.

## The Task at Hand

The `AviationData.csv` dataset contains information about aviation accidents, but it requires *cleaning* and *preprocessing* due to missing values, inconsistencies, and potential outliers. The cleaned data will help:
- Identify key patterns in accidents and their causes.
- Understand the impact of weather and other conditions on accidents.
- Prepare a high-quality dataset for detailed analysis.

## Goals

1) To understand the structure of the dataset.
2) Clean missing values and ensure consistency.
3) Perform feature engineering for better insights.
4) Aggregate and analyze key data points.
5) Save the cleaned dataset for future use.

## 1. Import libraries and load dataset

In [ ]:
# Start by Importing libraries commonly used for data analysis and visualization with their corresponding aliases
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


# Load the dataset
df = pd.read_csv('AviationData.csv', encoding='ISO-8859-1', low_memory=False)
df

: 

## 2. Data Overview

In [ ]:
print(df.head())
print(df.info())
print(df.describe())  # For numerical columns

#Total missing values in each column
missing_values = df.isnull().sum().sort_values(ascending=False)
print("\nMissing Values in Each Column:")
print(missing_values)

#Unique values in a few categorical columns
categorical_cols = df.select_dtypes(include=['object']).columns
for col in categorical_cols[:5]:  # Check the first 5 categorical columns
    print(f"\nUnique values in '{col}':")
    print(df[col].unique())

df.shape

## 3. Remove Irrelevant Columns

In [ ]:
#Get column names
print(df.columns)

# Drop specific irrelevant columns
df = df.drop(['Event.Id', 'Airport.Code', 'Accident.Number', 'FAR.Description', 'Schedule'],axis=1)
print(df)
print(df.shape)

## 4. Change Column Format

In [ ]:
#For the columns to look neater and increased readability replace the dots with whitespaces
df.columns = df.columns.str.replace(".", " ")
df.columns

## 5. Handle Missing Values

In [ ]:
#Check if the dataset contains missing values
print(df.isnull().sum().any())

#If so how many missing values do we have in each column?
df.isna().sum()

#Handle missing values in numeric using median
for column in df.select_dtypes(include=["number"]).columns:
     df[column] = df[column].fillna(df[column].median())

# Handle categorical columns with missing values using mode
for column in df.select_dtypes(exclude=["number"]).columns:
    df[column] = df[column].fillna(df[column].mode()[0])

#Confirm that no missing values left
df.isna().sum()

In [ ]:
#confirm that there are no missing values left from the entire datset
df.isnull().sum().any()

## 6. Check for Duplicates

In [ ]:
#duplicates
duplicates = df[df.duplicated()]
print(duplicates)

#remove duplicates
df = df.drop_duplicates()

#confirm removal
df.duplicated().sum()

## 7. Check for Outliers

In [ ]:
sns.boxplot(df)

## 9. Get Rid of Outliers

In [290]:
#Select numeric columns from the dataframe
numeric_columns = df.select_dtypes(include=["number"]).columns

#Calculate Q1, Q3 and IQR for the numeric columns
Q1 = df[numeric_columns].quantile(0.25)
Q3 = df[numeric_columns].quantile(0.75)
IQR = Q3-Q1

#Define lower bounds and upper bounds for outlier removal
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

#Filter data to keep only numbers within the IQR bounds
no_outliers = df[(df[numeric_columns] >= lower_bound) & (df[numeric_columns] <= upper_bound)].dropna()

In [ ]:
df

## 7. Parse and extract date information

In [ ]:
# Confirm the column names
#print(df.columns)

# Parse date columns
df['Event Date'] = pd.to_datetime(df['Event Date'], errors='coerce', dayfirst=False)
df['Publication Date'] = pd.to_datetime(df['Publication Date'], errors='coerce', dayfirst=True)

# Extract year, month, and day from Event.Date
df['Event Year'] = df['Event Date'].dt.year
df['Event Month'] = df['Event Date'].dt.month
df['Event Day'] = df['Event Date'].dt.day

print(df.head())
df.columns

## Aggregation

In [ ]:
# Combine injury columns into a single metric
df['Total Injuries'] = (
    df['Total Fatal Injuries'] +
    df['Total Serious Injuries'] +
    df['Total Minor Injuries'] +
    df['Total Uninjured']
)

# Compare the columns (use total_injuries instead of total.injuries.calculated)
print((df['Total Injuries'] == df['Total Injuries']).all())

# Standardize 'Weather.Condition' values
if 'Weather Condition' in df.columns:
    df['Weather Condition'] = df['Weather Condition'].replace({'UNK': 'Unknown'}).str.title()

# Check the updated output
print(df[['Total Injuries', 'Weather Condition']].head())

# Cap injury values at the 99th percentile
for col in ['Total Fatal Injuries', 'Total Serious Injuries', 'Total Minor Injuries', 'Total Injuries']:
    upper_limit = df[col].quantile(0.99)
    df[col] = np.where(df[col] > upper_limit, upper_limit, df[col])


## Weather Condition (Fill 'UNK' with 'Unknown'):

In [ ]:
df['Weather Condition'] = df['Weather Condition'].replace({'UNK': 'Unknown'}).str.title()
df['Weather Condition'] 

## Filling missing values in the 'Number.of.Engines' column with the median

In [ ]:
df['Number of Engines'] = df['Number of Engines'].fillna(df['Number of Engines'].median())
df['Number of Engines']

## Categorical Columns (Fill with Mode)

In [307]:
categorical_cols = df.select_dtypes(include=['object']).columns
for col in categorical_cols:
    df[col] = df[col].fillna(df[col].mode()[0])

## Top Aircraft Makes (Categorize Others as 'Other')

In [310]:
top_makes = df['Make'].value_counts().nlargest(10).index
df['Make'] = df['Make'].apply(lambda x: x if x in top_makes else 'Other')

In [ ]:
print(df['weather.condition'].value_counts())
print(df['injury.severity'].value_counts())

In [ ]:
print(df.shape)  # Check the number of rows and columns
print(df.head())  # View the first few rows of the DataFrame
df['Publication Date'] = df['Publication Date'].fillna(method='ffill')
df['Publication Date'] = df['Publication Date'].ffill()
df['Publication Date'] = df['Publication Date'].bfill() 

## Confirm the file's content

In [ ]:
# Remaining missing values
print(df.isnull().sum())

# Final structure
print(df.info())

## Save the Cleaned Dataset

In [319]:
# Save cleaned dataset
df.to_csv('Cleaned_AviationDataset.csv', index=False)

## Conclusion

This notebook involved cleaning and analyzing aviation data. Missing values were handled, features were engineered, and outliers were addressed to ensure data quality. The cleaned dataset is now ready for further analysis or modeling.

The cleaned dataset is saved and ready for further use.

## Next Steps

1) Conduct advanced analysis (e.g., predictive modeling or clustering).
2) Explore temporal trends in aviation incidents.
3) Investigate relationships between aircraft types and injury severity.
4) Use the cleaned data to come up with an interactive dashboard.